In [2]:
import MySQLdb
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode() # run at the start of every notebook
import plotly.plotly as py
py.sign_in('lemoga.inv', '5mjpk13cyv')
import cufflinks as cf
import pandas as pd
import numpy as np

In [3]:
usr = "root"
xxx = "jl201130674mg"
hst = "localhost"
adb = "viruses"

db = MySQLdb.connect(user=usr, passwd=xxx, host=hst, db=adb)

In [4]:
cursor = db.cursor()
query = ("SELECT accession, length "
         "FROM bioentry INNER JOIN biosequence "
         "WHERE bioentry.biodatabase_id=1 AND bioentry.bioentry_id=biosequence.bioentry_id "
         "ORDER BY length")
cursor.execute(query)

13999L

In [5]:
allVirusesLengths = []
allVirusesAccessions = []
for accession,length in cursor:
    allVirusesAccessions.append(accession)
    allVirusesLengths.append(length)
cursor.close()
print allVirusesLengths[0:10]

[324L, 338L, 340L, 341L, 369L, 383L, 401L, 405L, 457L, 467L]


In [6]:
cursor = db.cursor()
query = ("SELECT bioentry.accession, biosequence.length, COUNT(seqfeature.seqfeature_id)"
         "FROM bioentry INNER JOIN biosequence INNER JOIN seqfeature "
         "WHERE bioentry.bioentry_id=biosequence.bioentry_id AND bioentry.bioentry_id=seqfeature.bioentry_id "
         "AND bioentry.biodatabase_id=2 "
         "GROUP BY bioentry.accession "
         "ORDER BY biosequence.length")
cursor.execute(query)

1973L

In [7]:
phagesLengths = []
phagesAccessions = []
phagesNumProteins = []
for accession,length, proteins in cursor:
    phagesAccessions.append(accession)
    phagesLengths.append(length)
    phagesNumProteins.append(proteins)
cursor.close()

In [8]:
cursor = db.cursor()
query = ("SELECT bioentry.accession, biosequence.length, COUNT(seqfeature.seqfeature_id)"
         "FROM bioentry INNER JOIN biosequence INNER JOIN seqfeature "
         "WHERE bioentry.bioentry_id=biosequence.bioentry_id AND bioentry.bioentry_id=seqfeature.bioentry_id "
         "AND bioentry.biodatabase_id=3 "
         "GROUP BY bioentry.accession ")
cursor.execute(query)

12026L

In [9]:
virusesLengths = []
virusesAccessions = []
virusesNumProteins = []
for accession,length, proteins in cursor:
    virusesAccessions.append(accession)
    virusesLengths.append(length)
    virusesNumProteins.append(proteins)
cursor.close()

In [74]:
trace1 = go.Box(y = allVirusesLengths, x=["All"]*len(allVirusesLengths),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="All")
trace2 = go.Box(y = phagesLengths, x=["Phages"]*len(phagesLengths),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="Phages")
trace3 = go.Box(y = virusesLengths, x=["Viruses"]*len(virusesLengths),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="Viruses")
data = [trace1,trace2,trace3]
layout = go.Layout(title='Genomes Length', 
                   yaxis=dict(title="Length", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black'))
                  )
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [10]:
cursor = db.cursor()
query = ("SELECT DISTINCT(bioentry.accession), taxon.taxon_id "
         "FROM bioentry INNER JOIN taxon INNER JOIN taxon_name "
         "WHERE bioentry.biodatabase_id=1 "
             "AND bioentry.taxon_id=taxon.taxon_id "
             "AND taxon.taxon_id=taxon_name.taxon_id "
             "AND taxon_name.name_class='scientific name'")
cursor.execute(query)
accessions = []
taxons = []
for accession,taxon in cursor:
    accessions.append(accession)
    taxons.append(taxon)
cursor.close()
print len(accessions)

tree = []
for taxon in taxons:
    cursor = db.cursor()
    q = "call path_to_root_node(" + str(taxon) + ")"
    query = (q)
    cursor.execute(query)
    for t in cursor:
        tree.append(t)
    cursor.close()
print len(tree)

infoType = {"dsDNA":[], "dsRNA":[], "ssDNA":[], "ssRNA":[], "Retro-transcribing":[], "environmental":[]}
for i in range(len(accessions)):
    for key in infoType:
        if key in tree[i][0]:
            aux = infoType[key]
            aux.append(accessions[i])
            infoType[key] = aux
            continue

13999
13999


In [11]:
allTypes = []
for value in infoType.values():
    allTypes += value
print len(allTypes)
print len(set(allTypes))

infoType["none"] = list(set(allVirusesAccessions) - (set(allTypes)))
print len(infoType["none"])

13612
13585
414


In [12]:
infoTypeByEntity = {}
for key in infoType:
    genomes = infoType[key]
    newKey = key + "V"
    infoTypeByEntity[newKey] = []
    for genome in genomes:
        if genome in virusesAccessions:
            infoTypeByEntity[newKey].append(genome)
    
for key in infoType:
    genomes = infoType[key]
    newKey = key + "P"
    infoTypeByEntity[newKey] = []
    for genome in genomes:
        if genome in phagesAccessions:
            infoTypeByEntity[newKey].append(genome)
    
print infoTypeByEntity.keys()

['noneV', 'noneP', 'dsRNAV', 'environmentalP', 'ssRNAP', 'environmentalV', 'ssRNAV', 'Retro-transcribingV', 'Retro-transcribingP', 'ssDNAV', 'dsRNAP', 'ssDNAP', 'dsDNAV', 'dsDNAP']


In [13]:
allI = 0
for key in infoTypeByEntity:
    allI += len(infoTypeByEntity[key])
    print key, len(infoTypeByEntity[key])
print allI

noneV 359
noneP 55
dsRNAV 444
environmentalP 6
ssRNAP 26
environmentalV 796
ssRNAV 4663
Retro-transcribingV 2189
Retro-transcribingP 0
ssDNAV 1600
dsRNAP 0
ssDNAP 125
dsDNAV 1997
dsDNAP 1766
14026


In [ ]:
for key in infoTypeByEntity:
    accs = infoTypeByEntity[key] #accs stands for accessions
    for i in range(len(accs)):
        infoTypeByEntity[key][i] = allVirusesLengths[allVirusesAccessions.index(accs[i])]

allInfoType = 0
for key in infoTypeByEntity:
    allInfoType += len(infoTypeByEntity[key])
print allInfoType

In [17]:
for key in infoType:
    accs = infoType[key] #accs stands for accessions
    for i in range(len(accs)):
        infoType[key][i] = allVirusesLengths[allVirusesAccessions.index(accs[i])]

allInfoType = 0
for key in infoType:
    allInfoType += len(infoType[key])
print allInfoType

14026


In [19]:
trace1 = go.Box(y = infoType["dsDNA"], x=["dsDNA"]*len(infoType["dsDNA"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="dsDNA")
trace2 = go.Box(y = infoType["dsRNA"], x=["dsRNA"]*len(infoType["dsRNA"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="dsRNA")
trace3 = go.Box(y = infoType["ssDNA"], x=["ssDNA"]*len(infoType["ssDNA"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="ssDNA")
trace4 = go.Box(y = infoType["ssRNA"], x=["ssRNA"]*len(infoType["ssRNA"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="ssRNA")
trace5 = go.Box(y = infoType["Retro-transcribing"], x=["Retro-transcribing"]*len(infoType["Retro-transcribing"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="Retro-transcribing")
trace6 = go.Box(y = infoType["environmental"], x=["Environmental"]*len(infoType["environmental"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="Environmental")
trace7 = go.Box(y = infoType["none"], x=["None"]*len(infoType["none"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="None")
data = [trace1,trace2,trace3,trace4,trace5, trace6, trace7]
layout = go.Layout(title='Genomes Length', 
                   yaxis=dict(title="Length", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black'))
                  )
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [57]:

trace1 = go.Box(y = infoTypeByEntity["dsDNAV"], x=["dsDNAV"]*len(infoTypeByEntity["dsDNAV"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="Virus", marker=dict(color="green"))
trace2 = go.Box(y = infoTypeByEntity["dsDNAP"], x=["dsDNAP"]*len(infoTypeByEntity["dsDNAP"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="Phages", marker=dict(color="orange"))
trace3 = go.Box(y = infoTypeByEntity["ssDNAV"], x=["ssDNAV"]*len(infoTypeByEntity["ssDNAV"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="Virus", marker=dict(color="green"))
trace4 = go.Box(y = infoTypeByEntity["ssDNAP"], x=["ssDNAP"]*len(infoTypeByEntity["ssDNAP"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="Phages", marker=dict(color="orange"))
trace5 = go.Box(y = infoTypeByEntity["dsRNAV"], x=["dsRNAV"]*len(infoTypeByEntity["dsRNAV"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="dsRNAV", marker=dict(color="green"))
trace6 = go.Box(y = infoTypeByEntity["dsRNAP"], x=["dsRNAP"]*len(infoTypeByEntity["dsRNAP"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="dsRNAP", marker=dict(color="orange"))
trace7 = go.Box(y = infoTypeByEntity["ssRNAV"], x=["ssRNAV"]*len(infoTypeByEntity["ssRNAV"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="ssRNAV", marker=dict(color="green"))
trace8 = go.Box(y = infoTypeByEntity["ssRNAP"], x=["ssRNAP"]*len(infoTypeByEntity["ssRNAP"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="ssRNAP", marker=dict(color="orange"))
trace9 = go.Box(y = infoType["Retro-transcribing"], 
                x=["Retro-transcribing"]*len(infoType["Retro-transcribing"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="Retro-transcribing")
trace10 = go.Box(y = infoType["environmental"], 
                x=["environmental"]*len(infoType["environmental"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="environmental")
trace11 = go.Box(y = infoType["none"], 
                x=["None"]*len(infoType["none"]),
                boxpoints='all',jitter=0.1, pointpos=-1.5, name="None")


data = [trace1,trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9,trace10,trace11]
layout = go.Layout(title='Genomes Length', 
                   yaxis=dict(title="Length", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black')),
                   xaxis=dict(tickangle=90)
                  )
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [82]:
trace1 = go.Scatter(
    x=phagesLengths,
    y=phagesNumProteins,
    mode='markers',
    marker=dict(color='orange'),
    name='Phages',
    text=accessions
)
data = [trace1]
layout = go.Layout(title='Phages Genic Density',
                  yaxis=dict(title="Proteins Number", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black')),
                  xaxis=dict(title="Genome Length", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black')))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [99]:
#This plot was taken this points [(KJ713298:12,4200);(AB303562:69,26499);(NC006006:612,348532)]

xi = np.array([2435,348532])
yi = np.array([10,612])
m,b = np.polyfit(xi, yi, 1)
print m,b
xi = np.array([2000,350532])

trace1 = go.Scatter(
    x=phagesLengths,
    y=phagesNumProteins,
    mode='markers',
    marker=dict(color='orange'),
    name='Phages',
    text=accessions
)

trace2 = go.Scatter(
    x=xi,
    y=(m*xi)+b,
    line = dict(
        color = "red",
        width = 1,
        dash = 'dot'),
    name = "Theoretical limit"
)

annotation = go.Annotation(
                  x=3.5,
                  y=1.5,
                  text='Y = 0.0017X + 5.765',
                  showarrow=False,
                  font=go.Font(size=12)
                  )

data = [trace1, trace2]
layout = go.Layout(title='Phages Genic Density',
                  yaxis=dict(title="Proteins Number", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black')),
                  xaxis=dict(title="Genome Length", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black')),
                  annotations=[annotation])
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)


0.00173939675871 5.76456889254


In [83]:
trace1 = go.Scatter(
    x=virusesLengths,
    y=virusesNumProteins,
    mode='markers',
    marker=dict(color='green'),
    name='Phages',
    text=accessions
)
data = [trace1]
layout = go.Layout(title='Viruses Genic Density',
                  yaxis=dict(title="Proteins Number", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black')),
                  xaxis=dict(title="Genome Length", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black')))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [115]:
xi = np.array([341,1191693])
yi = np.array([3,1571])
m,b = np.polyfit(xi, yi, 1)
print m,b
xi = np.array([300,2500000])

trace1 = go.Scatter(
    x=virusesLengths,
    y=virusesNumProteins,
    mode='markers',
    marker=dict(color='green'),
    name='Phages',
    text=accessions
)
trace2 = go.Scatter(
    x=xi,
    y=(m*xi)+b,
    line = dict(
        color = "red",
        width = 1,
        dash = 'dot'),
    name = "Theoretical limit"
)

annotation = go.Annotation(
                  x=2.7,
                  y=1.2,
                  text='Y = 0.0010X + 2.650',
                  showarrow=False,
                  font=go.Font(size=12)
                  )

data = [trace1,trace2]
layout = go.Layout(title='Viruses Genic Density',
                  yaxis=dict(title="Proteins Number", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black')),
                  xaxis=dict(title="Genome Length", zeroline=False, type="log", autorange=True, 
                              tickfont=dict(size=14,color='black')),
                  annotations=[annotation])
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

0.00131615173349 2.55119225888
